Imports and Setup

In [2]:

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!pip -q install wandb
!pip -q install fiftyone
!pip -q install --upgrade pip
!pip -q install backports.zstd zstandard py7zr
!pip install numpy pandas torch torchvision torchaudio

import torch
import wandb
from pathlib import Path
import random
import numpy as np



ERROR: To modify pip, please run the following command:
C:\Users\const\anaconda3\python.exe -m pip -q install --upgrade pip


In [3]:
import sys
from pathlib import Path

project_root = Path.cwd().parent  # go from notebooks/ -> CLIP-Multimodal-Learning/
sys.path.insert(0, str(project_root))

print("CWD:", Path.cwd())
print("Using project root:", project_root)
print("src exists:", (project_root / "src").exists())

import os, random, time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb

from pathlib import Path
from torch.utils.data import DataLoader



from src.datasets import AssessmentPairs, train_val_split, AssessmentTorchDataset, stratified_subsample, class_counts

# from google.colab import drive
# drive.mount('/content/drive')

CWD: g:\My Drive\Hands-on-CV-Project2\notebooks
Using project root: g:\My Drive\Hands-on-CV-Project2
src exists: True


In [ ]:
# cd drive/MyDrive/Hands-on-CV-Project2

/content/drive/MyDrive/Hands-on-CV-Project2


In [4]:
def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_all(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

Load data

In [5]:






# ---------- settings ----------
PROJECT = "cilp-extended-assessment"
EMB = 200
LR = 1e-3
TEMP = 0.07

EPOCHS_CILP = 20
EPOCHS_PROJ = 20
EPOCHS_CLF  = 10

# Task 4 winner (based on your table): MaxPool
best_pool = "maxpool"   # <-- important for rubric (your Task 4 results favor maxpool)

Path("checkpoints").mkdir(exist_ok=True)

wandb.login()






wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account


wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\const\_netrc
wandb: Currently logged in as: constantin-auga (Conscht-Sht) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
from pathlib import Path
from torch.utils.data import DataLoader
from src.datasets import (
    AssessmentPairs, train_val_split, AssessmentTorchDataset,
    stratified_subsample, class_counts
)
ROOT = Path(r"G:\My Drive\Hands-on-CV-Project2\assessment")

pairs = AssessmentPairs(ROOT).load_pairs()

SUBSET_FRAC = 0.10
pairs_sub = stratified_subsample(pairs, frac=SUBSET_FRAC, seed=42)

train_pairs, val_pairs = train_val_split(pairs_sub, val_ratio=0.2, seed=42)

train_ds = AssessmentTorchDataset(train_pairs)
val_ds   = AssessmentTorchDataset(val_pairs)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,
                          num_workers=2, pin_memory=True, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False,
                          num_workers=2, pin_memory=True, drop_last=True)


print("ALL:", len(pairs), class_counts(pairs))
print("SUB:", len(pairs_sub), class_counts(pairs_sub))
print("train:", len(train_pairs), class_counts(train_pairs))
print("val  :", len(val_pairs), class_counts(val_pairs))

batch = next(iter(train_loader))
rgb_in_ch = batch["rgb"].shape[1]
lidar_in_ch = batch["lidar"].shape[1]
print("rgb_in_ch:", rgb_in_ch, "lidar_in_ch:", lidar_in_ch)


ALL: 19998 {'cubes': 9999, 'spheres': 9999}
SUB: 2000 {'cubes': 1000, 'spheres': 1000}
train: 1600 {'cubes': 810, 'spheres': 790}
val  : 400 {'cubes': 190, 'spheres': 210}
rgb_in_ch: 4 lidar_in_ch: 1


Train model

In [10]:
from src.models import Embedder, EmbedderStrided, CILP, Projector, EmbeddingClassifier
from src.training import train_cilp, train_projector, train_embedding_classifier, clip_contrastive_loss
# ============================================================
# 5.1 Contrastive pretraining (CILP)
# ============================================================
run = wandb.init(
    project=PROJECT,
    name=f"task5_cilp_contrastive_{best_pool}",
    config={
        "task": 5,
        "stage": "contrastive",
        "pool_type": best_pool,
        "lr": LR,
        "epochs": EPOCHS_CILP,
        "batch_size": train_loader.batch_size,
        "temperature": TEMP,
        "emb": EMB,
        "normalize_embeddings": True,
    },
)

# Use provided Embedder OR best from Task 4: your best is MaxPool Embedder
rgb_embed = Embedder(rgb_in_ch, emb_size=EMB, normalize=True).to(device)
lid_embed = Embedder(lidar_in_ch, emb_size=EMB, normalize=True).to(device)

cilp = CILP(rgb_embed, lid_embed, temperature=TEMP).to(device)

batch = next(iter(train_loader))
rgb = batch["rgb"].to(device).float()
lidar = batch["lidar"].to(device).float()

cilp.train()
opt = torch.optim.Adam(cilp.parameters(), lr=1e-3)

# snapshot weights
w0 = {k: v.detach().clone() for k, v in cilp.state_dict().items()}

loss1 = clip_contrastive_loss(cilp(rgb, lidar))
opt.zero_grad(set_to_none=True)
loss1.backward()

# grad check
gmax = 0.0
for p in cilp.parameters():
    if p.grad is not None:
        gmax = max(gmax, p.grad.detach().abs().max().item())
print("loss1:", float(loss1.item()), "max|grad|:", gmax)

opt.step()

# weight change check
w1 = cilp.state_dict()
dmax = 0.0
for k in w0:
    dmax = max(dmax, (w1[k] - w0[k]).abs().mean().item())
print("max mean |Δw|:", dmax)

with torch.no_grad():
    loss2 = clip_contrastive_loss(cilp(rgb, lidar)).item()
print("loss2:", loss2)


# train_cilp should log train_loss + val_loss each epoch (contrastive loss)
cilp_out = train_cilp(
    cilp,
    train_loader,
    val_loader,
    device=device,
    epochs=EPOCHS_CILP,
    lr=LR,
    wandb_run=run,
)

# If your function returns final val loss, log it explicitly too
if isinstance(cilp_out, dict) and "val_loss_final" in cilp_out:
    wandb.log({"val_loss_final": cilp_out["val_loss_final"]})

run.finish()

torch.save(cilp.state_dict(), "checkpoints/cilp_contrastive.pt")


loss1: 3.4657914638519287 max|grad|: 0.0026178243570029736
max mean |Δw|: 0.0009999275207519531
loss2: 3.465756416320801
epoch 01 | train 3.4520 | val 3.3981
epoch 02 | train 3.3967 | val 3.2909
epoch 03 | train 3.1860 | val 3.1332
epoch 04 | train 3.0239 | val 3.0198
epoch 05 | train 2.9560 | val 2.9229
epoch 06 | train 2.8548 | val 2.8096
epoch 07 | train 2.7159 | val 2.9039
epoch 08 | train 2.4743 | val 2.5498
epoch 09 | train 2.3400 | val 2.4068
epoch 10 | train 2.2271 | val 2.6970
epoch 11 | train 2.1645 | val 2.2617
epoch 12 | train 2.0345 | val 2.1690
epoch 13 | train 1.9370 | val 2.2070
epoch 14 | train 1.8426 | val 2.0336
epoch 15 | train 1.6993 | val 2.1305
epoch 16 | train 1.5254 | val 1.6627
epoch 17 | train 1.2064 | val 1.7565
epoch 18 | train 1.2513 | val 1.8394
epoch 19 | train 0.9991 | val 1.5257
epoch 20 | train 0.8661 | val 1.5236


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/contrastive_loss,██▇▇▇▆▆▅▅▅▅▄▄▄▃▃▂▂▁▁
val/contrastive_loss,██▇▇▆▆▆▅▄▅▄▃▄▃▃▂▂▂▁▁
epoch,20
lr,0.001
train/contrastive_loss,0.8661
val/contrastive_loss,1.52364


Train Projetor

In [11]:
# ============================================================
# 5.2 Cross-modal projector (RGB -> LiDAR embedding space)
# Freeze CILP encoders so ONLY projector learns the mapping
# ============================================================
for p in cilp.parameters():
    p.requires_grad = False
cilp.eval()

run = wandb.init(
    project=PROJECT,
    name=f"task5_projector_rgb2lid_{best_pool}",
    config={
        "task": 5,
        "stage": "projector",
        "pool_type": best_pool,
        "lr": LR,
        "epochs": EPOCHS_PROJ,
        "dim": EMB,
        "loss": "mse(proj(z_rgb), z_lidar)",
        "encoders_frozen": True,
    },
)

proj = Projector(dim=EMB).to(device)

# train_projector should minimize:
# z_rgb = cilp.rgb(rgb)
# z_lid = cilp.lidar(lidar)
# z_hat = proj(z_rgb)
# loss = MSE(z_hat, z_lid)
proj_out = train_projector(
    cilp,
    proj,
    train_loader,
    val_loader,
    device=device,
    epochs=EPOCHS_PROJ,
    lr=LR,
    wandb_run=run,
)

if isinstance(proj_out, dict) and "val_mse_final" in proj_out:
    wandb.log({"val_mse_final": proj_out["val_mse_final"]})

run.finish()

torch.save(proj.state_dict(), "checkpoints/projector_rgb_to_lidar.pt")

epoch 01 | train mse 0.0016 | val mse 0.0009
epoch 02 | train mse 0.0005 | val mse 0.0009
epoch 03 | train mse 0.0004 | val mse 0.0009
epoch 04 | train mse 0.0004 | val mse 0.0009
epoch 05 | train mse 0.0004 | val mse 0.0009
epoch 06 | train mse 0.0004 | val mse 0.0009
epoch 07 | train mse 0.0004 | val mse 0.0009
epoch 08 | train mse 0.0004 | val mse 0.0009
epoch 09 | train mse 0.0004 | val mse 0.0009
epoch 10 | train mse 0.0004 | val mse 0.0009
epoch 11 | train mse 0.0004 | val mse 0.0009
epoch 12 | train mse 0.0004 | val mse 0.0009
epoch 13 | train mse 0.0004 | val mse 0.0009
epoch 14 | train mse 0.0004 | val mse 0.0009
epoch 15 | train mse 0.0004 | val mse 0.0008
epoch 16 | train mse 0.0004 | val mse 0.0009
epoch 17 | train mse 0.0004 | val mse 0.0009
epoch 18 | train mse 0.0004 | val mse 0.0009
epoch 19 | train mse 0.0004 | val mse 0.0009
epoch 20 | train mse 0.0004 | val mse 0.0009


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train/projector_mse,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/projector_mse,█▄▅▃▅▄▁▄▃▃▂▅▂▃▁▃▂▄▂▂
epoch,20
train/projector_mse,0.0004
val/projector_mse,0.00086


Train final Classifier

In [ ]:
# # ============================================================
# # 5.3 Final classifier (predict y from projected RGB embeddings)
# # Freeze projector too (clean evaluation)
# # ============================================================
import importlib
import src.visualization as vis
importlib.reload(vis)
from src.visualization import log_task5_predictions
for p in proj.parameters():
    p.requires_grad = False
proj.eval()

run = wandb.init(
    project=PROJECT,
    name=f"task5_classifier_rgb2lid_{best_pool}",
    config={
        "task": 5,
        "stage": "classifier",
        "pool_type": best_pool,
        "lr": LR,
        "epochs": EPOCHS_CLF,
        "input": "proj(cilp.rgb(rgb))",
        "encoders_frozen": True,
        "projector_frozen": True,
    },
)

clf = EmbeddingClassifier(dim=EMB).to(device)

# train_embedding_classifier should train:
# features = proj(cilp.rgb(rgb))
# labels = y
clf_out = train_embedding_classifier(
    cilp,
    proj,
    clf,
    train_loader,
    val_loader,
    device=device,
    epochs=EPOCHS_CLF,
    lr=LR,
    wandb_run=run,
)



log_task5_predictions(cilp, proj, clf, val_loader, device, run, n=5)

if isinstance(clf_out, dict) and "val_acc_final" in clf_out:
    wandb.log({"val_acc_final": clf_out["val_acc_final"]})

run.finish()

torch.save(clf.state_dict(), "checkpoints/classifier_rgb_to_lidar.pt")

epoch,▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█
train/clf_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/acc,▁▄▄▇▇▇█▇▇▅▇▇▇▇▇█▇▇▇▅
val/clf_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂
epoch,10
train/clf_loss,0.0218
val/acc,0.98438
val/clf_loss,0.0384


Log Matrix

In [30]:
import src.visualization as vis
importlib.reload(vis)
from src.visualization import log_similarity_matrix_artifact

import wandb



# start or reuse a run
import wandb
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False,
                          num_workers=2, pin_memory=True, drop_last=True)
PROJECT_NAME = "Hands-on-CV-Project2"  
run = wandb.init(project=PROJECT_NAME, name="task5_similarity", reinit=True)

run = wandb.init(project=PROJECT_NAME, name="task5_similarity", reinit=True)

# load your trained weights if not already loaded
# cilp.load_state_dict(torch.load("checkpoints/xxx.pt", map_location=device))
# cilp.to(device).eval()

S = log_similarity_matrix_artifact(
    cilp, val_loader, device, run,
    n=32,
    artifact_name="task5_image_lidar_similarity",
    artifact_type="analysis",
)

wandb.finish()


Testing

In [19]:


# ============================================================
# Required: test on at least 5 validation batches
# ============================================================
@torch.no_grad()
def test_5_batches(cilp, proj, clf, loader, device):
    cilp.eval(); proj.eval(); clf.eval()
    correct, total = 0, 0
    it = iter(loader)
    for _ in range(5):
        batch = next(it)
        rgb = batch["rgb"].to(device)
        y   = batch["y"].to(device)

        z = cilp.rgb(rgb)
        z_hat = proj(z)
        logits = clf(z_hat)
        pred = torch.argmax(logits, dim=1)

        correct += (pred == y).sum().item()
        total += y.numel()

    acc = correct / total
    print("Accuracy over 5 val batches:", acc)
    return acc

acc5 = test_5_batches(cilp, proj, clf, val_loader, device)

# optional: log the 5-batch check too
wandb.init(project=PROJECT, name=f"task5_5batch_check_{best_pool}", config={"task": 5, "stage": "5batch_check"})
wandb.log({"val_acc_5batches": acc5})
wandb.finish()

Accuracy over 5 val batches: 0.975


val_acc_5batches,▁
val_acc_5batches,0.975


Save the checkpoint

In [20]:
from pathlib import Path
Path("checkpoints").mkdir(exist_ok=True)

torch.save(cilp.state_dict(), "checkpoints/cilp_contrastive.pt")
torch.save(proj.state_dict(), "checkpoints/projector_rgb_to_lidar.pt")
torch.save(clf.state_dict(), "checkpoints/classifier_rgb_to_lidar.pt")
